In [131]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import requests
# Ignore all warnings
warnings.filterwarnings("ignore")

In [132]:
def get_data(symbol):
    resp = requests.get(
                
                'http://192.168.0.172:8080/api/data'
                ,
                            {
                                'sym': symbol.upper()+'c1',
                                'from': '2000-01-01', #'from' and 'to' parameters are requesting all data in our db
                                'to': '2024-03-28'
                            })
    data = resp.json()
    df = pd.DataFrame(data)
    df = df[['timestamp' , 'open', 'high', 'low', 'close', 'volume' ]]
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['timestamp'] = df['timestamp'].dt.strftime("%m/%d/%Y")
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    loc_index = df.columns.get_loc('timestamp') + 1
    df.insert(loc=loc_index,column='Week',value=df['timestamp'].dt.strftime("%V"))
    df['Week'] = pd.to_numeric(df['Week'], errors='coerce')

    loc_index = df.columns.get_loc('timestamp') + 2
    df.insert(loc=loc_index,column='Month',value=df['timestamp'].dt.month)
    return df


In [133]:
df = get_data('ZW')

In [134]:
def df_volatility(df , user_input , TRADING_DAYS):
    log_returns_avg = np.log(df['close']/df['close'].shift(1)).dropna()

    df.loc[:, 'daily_vol_avg'] = log_returns_avg.rolling(window=TRADING_DAYS).std()
    df.loc[:, 'annual_vol_'+user_input] = log_returns_avg.rolling(window=TRADING_DAYS).std() * np.sqrt(252) * 100
    #drop null reset index
    df = df.dropna(inplace=False).reset_index(drop=True)
    return df    

In [135]:
df = df_volatility(df , 'ZW' , 14)

In [136]:
df['sma'] = df['close'].rolling(window=14).mean()

In [137]:
df.dropna(inplace=True)

In [138]:
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Week', 'Month', 'Daily_Volatility', 'Annual_Volatility', 'SMA']

In [172]:
df['upper'] = df['SMA'] + 2*df['Annual_Volatility']
df['lower'] = df['SMA'] - 2*df['Annual_Volatility']

In [173]:
df['widht'] = (df['upper'] - df['lower'])/df['SMA']*100

In [174]:
df.index = df['Date']

In [175]:
df

,Date,Open,High,Low,Close,Volume,Week,Month,Daily_Volatility,Annual_Volatility,SMA,upper,lower,widht,widht2
Date,,,,,,,,,,,,,,,
2000-02-10,2000-02-10,6,2,265.00,273.50,265.00,271.75,35791,0.013781,21.876841,260.821429,304.575110,217.067748,33.550680,78.977154
2000-02-11,2000-02-11,6,2,272.75,273.50,267.50,269.50,35626,0.013973,22.181733,261.071429,305.434894,216.707963,33.985691,78.015331
2000-02-14,2000-02-14,7,2,269.00,272.75,268.00,268.75,12204,0.013923,22.101726,261.357143,305.560595,217.153690,33.826091,75.773682
2000-02-15,2000-02-15,7,2,269.50,269.50,265.00,266.00,16493,0.013234,21.008633,261.785714,303.802981,219.768448,32.100504,68.359799
2000-02-16,2000-02-16,7,2,266.00,268.00,263.25,265.25,21644,0.012831,20.367820,261.910714,302.646355,221.175074,31.106509,64.948300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21,2024-03-21,12,3,546.00,552.50,540.25,546.75,43716,0.018483,29.341632,542.785714,601.468979,484.102450,21.622995,66.553758
2024-03-22,2024-03-22,12,3,548.50,559.25,539.50,554.75,58439,0.018681,29.655854,542.125000,601.436708,482.813292,21.881193,78.872883
2024-03-25,2024-03-25,13,3,559.00,567.00,549.50,555.00,63147,0.017561,27.877556,542.410714,598.165827,486.655602,20.558264,94.102298


In [176]:
fig = px.line(df, x='Date', y=['Close']) 
fig.add_trace(go.Scatter(x=df.index, y=df['upper' ], mode='lines', name='Upper Band'))
fig.add_trace(go.Scatter(x=df.index, y=df['lower' ], mode='lines', name='Lower Band'))
fig.show()

In [177]:
df['widht2'] = (df['Close'] - df['lower'])/(df['upper'] - df['lower'])*100

In [178]:

#chart for volatility and close price
def line_chart_close_and_volatility(df, a , TRADING_DAYS):
        

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
        go.Scatter(x=df['Date'], y=df['Annual_Volatility'], name="annual_vol "+a,
                   yaxis='y1'),  # Specify y-axis using yaxis property
    )
    fig.add_trace(
        go.Scatter(x=df['Date'], y=df['Close'], name="close", 
                   yaxis='y2'),  # Specify y-axis using yaxis property
    )
    fig.update_layout(
        title_text="Annualize Volatility of "+a+" with Close Price",
    )
    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="<b>Volatility of "+a+"</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>Close Price</b> ", secondary_y=True)
    fig.update_layout(hovermode="x unified" , showlegend=True)
    # fig.update_traces(hovertemplate='Close: %{y}<br>Volatility: %{y1}')
    fig.show()

In [179]:
line_chart_close_and_volatility(df, 'ZW', 14)

In [180]:
fig = px.line(df, x='Date', y=['widht2' ])
fig.show()

In [181]:
fig = px.line(df, x='Date', y=['Close', 'upper' ,'lower'])
fig.show()

In [186]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['Annual_Volatility'], name="annual_vol ",
                yaxis='y1'),  # Specify y-axis using yaxis property
)
fig.add_trace(
    go.Scatter(x=df['Date'], y=df['width'], name="close", 
                yaxis='y2'),  # Specify y-axis using yaxis property
)
fig.update_layout(
    title_text="Annualize Volatility of "+" with Close Price",
)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="<b>Volatility of "+"</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Close Price</b> ", secondary_y=True)
fig.update_layout(hovermode="x unified" , showlegend=True)

KeyError: 'width'

In [1]:
import datetime
import requests
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [2]:
def get_data():
    resp = requests.get(
                
                'http://192.168.0.172:8080/api/v1/data/settlement?symbols=MESc1&from=2023-01-01&to=2024-04-17')
        
    data = resp.json()
    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['timestamp'] = df['timestamp'].dt.strftime("%m/%d/%Y")
    df['timestamp'] = pd.to_datetime(df['timestamp'])


    loc_index = df.columns.get_loc('timestamp') + 1
    df.insert(loc=loc_index,column='Month',value=df['timestamp'].dt.month)
    loc_index = df.columns.get_loc('timestamp') + 2
    df.insert(loc=loc_index,column='Year',value=df['timestamp'].dt.year)
    return df

In [3]:
df = get_data()

In [4]:
df

,timestamp,Month,Year,sym,settlement
0,2023-01-03,1,2023,MESc1,3846.00
1,2023-01-04,1,2023,MESc1,3874.50
2,2023-01-05,1,2023,MESc1,3829.00
3,2023-01-06,1,2023,MESc1,3915.50
4,2023-01-09,1,2023,MESc1,3913.75
...,...,...,...,...,...
320,2024-04-11,4,2024,MESc1,5243.25
321,2024-04-12,4,2024,MESc1,5167.50
322,2024-04-15,4,2024,MESc1,5104.00
323,2024-04-16,4,2024,MESc1,5092.50


In [5]:
def round( n ): 
  
    # Smaller multiple 
    a = (n // 10) * 10
      
    # Larger multiple 
    b = a + 10
      
    # Return of closest of two 
    return (b if n - a > b - n else a) 


month_code = dict( {
    1: 'F',
    2: 'G',
    3: 'H',
    4: 'J',
    5: 'K',
    6: 'M',
    7: 'N',
    8: 'Q',
    9: 'U',
    10: 'V',
    11: 'X',
    12: 'Z'
})

In [8]:
df

,timestamp,Month,Year,sym,settlement,round_settlement,month_code,last_digit_year,hundred
0,2023-01-03,1,2023,MESc1,3846.00,3850,F,23,00
1,2023-01-04,1,2023,MESc1,3874.50,3870,F,23,00
2,2023-01-05,1,2023,MESc1,3829.00,3830,F,23,00
3,2023-01-06,1,2023,MESc1,3915.50,3920,F,23,00
4,2023-01-09,1,2023,MESc1,3913.75,3910,F,23,00
...,...,...,...,...,...,...,...,...,...
320,2024-04-11,4,2024,MESc1,5243.25,5240,J,24,00
321,2024-04-12,4,2024,MESc1,5167.50,5170,J,24,00
322,2024-04-15,4,2024,MESc1,5104.00,5100,J,24,00
323,2024-04-16,4,2024,MESc1,5092.50,5090,J,24,00


In [9]:
def get_symbol(df):
    df['call_symbol'] = ("@"+'EW'+df['month_code']+df['last_digit_year']+"C"+df['round_settlement']+df['hundred']).astype(str)   
    df['put_symbol'] = ("@"+'EW'+df['month_code']+df['last_digit_year']+"P"+df['round_settlement']+df['hundred']).astype(str)
    return df   

In [10]:
df = get_symbol(df)

In [11]:
df = df[['timestamp' , 'call_symbol', 'put_symbol']]

In [12]:
df['start_time'] = df['timestamp']
df['end_time'] = df['timestamp'].shift(-1)
df

C:\Users\IBISP\AppData\Local\Temp\ipykernel_31656\4051059965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['start_time'] = df['timestamp']
C:\Users\IBISP\AppData\Local\Temp\ipykernel_31656\4051059965.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['end_time'] = df['timestamp'].shift(-1)


,timestamp,call_symbol,put_symbol,start_time,end_time
0,2023-01-03,@EWF23C385000,@EWF23P385000,2023-01-03,2023-01-04
1,2023-01-04,@EWF23C387000,@EWF23P387000,2023-01-04,2023-01-05
2,2023-01-05,@EWF23C383000,@EWF23P383000,2023-01-05,2023-01-06
3,2023-01-06,@EWF23C392000,@EWF23P392000,2023-01-06,2023-01-09
4,2023-01-09,@EWF23C391000,@EWF23P391000,2023-01-09,2023-01-10
...,...,...,...,...,...
320,2024-04-11,@EWJ24C524000,@EWJ24P524000,2024-04-11,2024-04-12
321,2024-04-12,@EWJ24C517000,@EWJ24P517000,2024-04-12,2024-04-15
322,2024-04-15,@EWJ24C510000,@EWJ24P510000,2024-04-15,2024-04-16
323,2024-04-16,@EWJ24C509000,@EWJ24P509000,2024-04-16,2024-04-17


In [13]:
df = df.dropna()

In [14]:
df.to_csv('one_year_data_esp.csv', index=True)